# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

# Please use jupyter notebook to check this homework. Jupyter lab is not working

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
citydata = pd.read_csv("citydata.csv")
#len(citydata["City"].unique())
citydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        540 non-null    object 
 1   Cloudiness  540 non-null    int64  
 2   Country     538 non-null    object 
 3   Date        540 non-null    int64  
 4   Humidity    540 non-null    int64  
 5   Lat         540 non-null    float64
 6   Lng         540 non-null    float64
 7   Max Temp    540 non-null    float64
 8   Wind Speed  540 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 38.1+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = citydata[["Lat", "Lng"]]
humidity = citydata["Humidity"].astype(float)
maxhumidity=humidity.max()

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius=4)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# new dataframe fitting weather criteria
new_df=citydata.loc[(citydata["Max Temp"]> 70) & (citydata["Max Temp"] < 80) & (citydata["Wind Speed"] < 10) & (citydata ["Cloudiness"]==0)] 
new_df = new_df.dropna(how='any')
new_df=new_df.reset_index(drop=True) 
new_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Pingdingshan,0,CN,1587272650,38,33.74,113.30,71.91,9.69
1,Port Elizabeth,0,ZA,1587272803,37,-33.92,25.57,72.54,8.10
2,San Juan,0,AR,1587272916,38,-31.54,-68.54,70.23,6.62
3,Bereket,0,TM,1587273039,26,39.24,55.52,78.98,8.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# add Hotel Name into the dataframe
hotel_df=new_df
hotel_df["Hotel Name"]=""
hotel_df

# set an empty hotel list
hotel = []

# set for loop to use parameters to look for nearby hotels
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    try:
        hotel.append(response['results'][0]['name'])
    except:
        hotel.append("")
# add the hotel list into the column of "Hotel Name"
hotel_df["Hotel Name"] = hotel

#drop any NA
hotel_df = hotel_df.dropna(how='any')

# display the results
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Pingdingshan,0,CN,1587272650,38,33.74,113.30,71.91,9.69,Pingdingshan
1,Port Elizabeth,0,ZA,1587272803,37,-33.92,25.57,72.54,8.10,Port Elizabeth
2,San Juan,0,AR,1587272916,38,-31.54,-68.54,70.23,6.62,San Juan
3,Bereket,0,TM,1587273039,26,39.24,55.52,78.98,8.34,Bereket


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

# please use jupyter notebook to run this code, the jupyter lab did not work